# 1. Основы PySpark

## 1.1. Введение и установка

PySpark — это Python API для Apache Spark — высокопроизводительной распределённой платформы для обработки больших данных. Позволяет писать параллельные программы, которые работают на кластерах, но при этом использовать знакомый Python. Spark работает с большими объёмами данных, поддерживает SQL, машинное обучение, потоковую обработку. Jupyter Notebook — удобный инструмент для интерактивного написания и запуска кода, идеально подходит для обучения и прототипирования.